In [2]:
library(dplyr)

# Read in data

In [3]:
data=read.csv('/dagher/dagher11/filip/UPF/data/dataset_excluded.csv', quote='"')

# Coalesce food data columns

In [42]:
data_food=data.frame(data[grep('intake',colnames(data))],data[grep('buttermargarine',colnames(data))],
                     data[grep('coffee',colnames(data))], 'eid'=data$eid)

In [9]:
coalesce_set <- function(data, prefix) {
  cols <- grep(paste0("^", prefix), names(data), value = TRUE)
  ordered_cols <- sort(cols, decreasing = TRUE) 
  print(ordered_cols)
  result <- coalesce(!!!as.list(data[ordered_cols]))
  return(result)
}

In [5]:
unique_prefixes <- unique(sub("_[0-9]+\\..*", "", names(data_food)))

for (prefix in unique_prefixes[-1]) {
  data[[paste0(prefix, "_coalesced")]] <- coalesce_set(data_food, prefix)
}

# Create df with only coalesced data and save it for further analyses

In [6]:
data_food=data[grep('coalesced',colnames(data))]

In [7]:
write.table(data_food,'/dagher/dagher11/filip/UPF/data/coalesced_intake.csv',sep=',',quote=TRUE)

# Split data into 5 dataframes for separate NOVA calculations

In [37]:
data=subset(data, !is.na(data$uk_biobank_assessment_centre_54.2.0))

In [47]:
split_timepoints <- function(df) {
  # For each timepoint
  for(t in 0:4) {
    # Get columns for this timepoint (ending in .t.0) plus eid
    cols <- c("eid", grep(paste0("\\.", t, "\\.0$"), names(df), value=TRUE))
    
    # Create new dataframe for this timepoint
    new_df <- df[, cols]
    
    # Remove the timepoint suffix from column names, but keep 'eid' as is
    names(new_df)[-1] <- sub(paste0("\\.", t, "\\.0$"), "", names(new_df)[-1])
    
    # Assign to global environment
    assign(paste0("df_t", t), new_df, envir = .GlobalEnv)
  }
}




In [48]:
split_timepoints(data_food)

In [49]:
write.table(df_t0,'/dagher/dagher11/filip/UPF/data/coalesced_intake_0.csv',sep=',',quote=TRUE)
write.table(df_t1,'/dagher/dagher11/filip/UPF/data/coalesced_intake_1.csv',sep=',',quote=TRUE)
write.table(df_t2,'/dagher/dagher11/filip/UPF/data/coalesced_intake_2.csv',sep=',',quote=TRUE)
write.table(df_t3,'/dagher/dagher11/filip/UPF/data/coalesced_intake_3.csv',sep=',',quote=TRUE)
write.table(df_t4,'/dagher/dagher11/filip/UPF/data/coalesced_intake_4.csv',sep=',',quote=TRUE)

In [44]:
write.table(data_food,'/dagher/dagher11/filip/UPF/data/coalesced_intake_all_timepoints.csv',sep=',',quote=TRUE)